In [27]:
import random
import bisect
from tkinter import *
import math

m = 5
deleted_nodes = []

def init_chord(node, finger_table, physical_nodes):
    finger_table[node] = [-1] * m * 2 #successor & predecessor list [0:m] predecessor, [m+1:2m] successor
    for i in range(m):
        table_entry = (node + 2 ** i) % 2 ** m
        finger_table[node][i] = table_entry #successor
        finger_table[node][i+m] = node #predecessor
    stabilize_chord(physical_nodes, finger_table)
    return finger_table
        
def add_finger_table(node, physical_nodes, finger_table):
    finger_table[node] = [-1] * m * 2
    for i in range(m):
        table_entry = (node + 2 ** i) % 2 ** m
        finger_table[node][i] = table_entry
        finger_table[node][i+m] = min([j for j in physical_nodes if j >= table_entry], default=-1) #successor
        if finger_table[node][i+m] == -1:
            finger_table[node][i+m] = physical_nodes[0]
    return finger_table

def stabilize_chord(physical_nodes, finger_table):
    for key in finger_table:
        for i in range(m):
            finger_table[key][i+m] = min([j for j in physical_nodes if j >= finger_table[key][i]], default=-1)
            if finger_table[key][i+m] == -1:
                finger_table[key][i+m] = physical_nodes[0]
    #print("finger "+str(finger_table))
    return finger_table

def add_node(physical_nodes):
    new_node = random.randint(0, 2**m-1)
    print("Randomly selected node in between 0 and "+str(2**m-1) +" is: "+str(new_node))
    if new_node in physical_nodes:
        new_node = -1
    return new_node

def delete_node(physical_nodes, finger_table, node):
    if node not in physical_nodes:
        return -2
    physical_nodes.remove(node)
    del finger_table[node]
    deleted_nodes.append(node)
    return stabilize_chord(physical_nodes, finger_table)

def rec_lookup(init_node, node, physical_nodes, finger_table, flag):
    flag = 0
    index = 2 * m -1  #total successors and predecessors of any node
    ind = physical_nodes.index(init_node)
    
    if init_node == physical_nodes[-1]:
        if node > init_node and node < 2 ** m:
            print(str(node)+" is located @Node: "+str(physical_nodes[0]))
            return 1 
    else:
        if init_node < node < physical_nodes[ind+1]:
            print(str(node)+ " is in between "+str(physical_nodes[ind]) 
          + " and its Successor " + str(physical_nodes[ind+1]))
            print(str(node) + " is located @Node: "+str(physical_nodes[ind+1]))
            return 1

    print("Finger table of "+str(init_node) + "- "+str(finger_table[init_node][m:index+1]))
    #predecessor_list = [finger_table[init_node].index(i) for i in finger_table[init_node] if i <= node]
    predecessor = min(finger_table[init_node][m:index+1])
    for i in range(index, m-1, -1):
        table_entry = finger_table[init_node][i]

        if table_entry < node:
            if table_entry > predecessor:
                predecessor = table_entry
            else:
                continue
    
            #if init_node < table_entry or node < init_node:
                #print(str(finger_table[init_node][m:index]))
                #print(str(node)+" not found in the finger table of "+str(init_node))
                #print("Lookup started @Node-" + str(table_entry))
                #flag = rec_lookup(table_entry, node, physical_nodes, finger_table, flag)
                #return flag
    ind_predecessor = physical_nodes.index(predecessor)
    #print("Successor- "+str(physical_nodes[ind+1]))
    print(str(node)+ " is not in between "+str(physical_nodes[ind]) 
          + " and its Successor " + str(physical_nodes[ind+1]))
    print("Finding Closest predecessor of " + str(node) + "...")
    print("Closest predecessor found. It is "+str(predecessor))
    print("Starting to lookup at "+str(predecessor))
    flag = rec_lookup(predecessor, node, physical_nodes, finger_table, flag)
    return flag

def lookup(physical_nodes, finger_table, node): #lookup happens recursively in successors only
    if node < 0 or node >= 2**m:
        return -1
    if node in deleted_nodes:
        print("This node has been deleted. Try another one.\n")
        return -2
    if node in physical_nodes:
        print('\n'+str(node)+" is already in the physical node list. So, it is located within Node "+str(node))
        return 1

    init_node = random.choice(physical_nodes)
    print("\nLookup started randomly @Node "+str(init_node))
    #print("Finger table of "+ str(init_node) + "- "+str(finger_table[init_node][m:2**m]))
    flag = rec_lookup(init_node, node, physical_nodes, finger_table, 0)
    return flag

def show(physical_nodes):
    window = Tk()
    total_nodes = 2**m
    
    height = window.winfo_screenheight()
    width = window.winfo_screenwidth() 

    canvas = Canvas(window, bg="white", height=height, width=width)
    canvas.grid()
    
    x0 = 50
    y0 = height-100
    x1 = height-100
    y1 = 50

    circle = canvas.create_oval(x0, y0, x1, y1, width=2) #chord ring, square shape

    r = (x1 - x0)//2
    a = x0 + r
    b = y1 + r
    offset = 15

    for i in range(total_nodes):
        degree = i * 360/total_nodes
        x = a+r*math.sin(math.radians(degree)) #x coordinate of center
        y = b+r*math.cos(math.radians(degree)) #y coordinate of center
        x0 = x-offset
        y0 = y+offset
        x1 = x+offset
        y1 = y-offset
        if i not in physical_nodes:
            canvas.create_oval(x0, y0, x1, y1, fill="white", width=1)
        else:
            canvas.create_oval(x0, y0, x1, y1, fill="lawngreen", width=1)
        canvas.create_text(x,y,text=i)
        
    def close_window():
        window.destroy()
    window.protocol("WM_DELETE_WINDOW", close_window)
    window.title("CHORD")
    window.mainloop()
              

if __name__ == '__main__':
    init_node = 0
    physical_nodes = [init_node]
    
    #test_nodes = [1, 21, 9, 25, 26, 7, 19, 2, 3, 31]
    
    finger_table = {}
    finger_table = init_chord(init_node, finger_table, physical_nodes)
    
    print("This is a CHORD system. It is an example of Structured Peer-to-Peer architecture.\n")
    print("In this implementation, you can add, delete, lookup any physical/logical node, or view the CHORD.\n")
    print("As per instruction, initially there is one node in the CHORD (which is " + str(init_node) + ").\n")
    print("You can delete any physical node from the CHORD till there is at least one node left.\n")
    print("In the CHORD visualization, the green nodes are the physical nodes.\n")
    print("Following is the instruction for performing add/delete/lookup operations.\nLet's start!\n")
    
    while(True):
        print("\n"+"*"*50)
        print("Press:\n1 Add a random node\n2 Delete a node\n3 Lookup any node\n4 View CHORD\n5 Exit the program")
        print("*"*50)
        print("Enter your choice: ")
        choice = input()
        if choice.isnumeric() == True:
            choice = int(choice)
        else:
            print("Please enter a valid choice.\n")
            continue
        if choice == 1:
            print("*"*50)
            new_node = add_node(physical_nodes)
            if new_node != -1:
                if new_node in deleted_nodes:
                    deleted_nodes.remove(new_node)
                bisect.insort(physical_nodes, new_node)
                add_finger_table(new_node, physical_nodes, finger_table)
                finger_table = stabilize_chord(physical_nodes, finger_table)
                print('\n' + str(new_node) + ' successfully added to the CHORD!')
                print("\nPhysical nodes in CHORD till now: "+str(physical_nodes))
            else:
                print("\n\nRandomly selected already exists in the CHORD. Try again.")
        elif choice == 2:
            print("*"*50)
            if len(physical_nodes) == 1:
                print("\n\nSorry only one node left. Add another node first.")
                continue
            print("\nPhysical nodes in CHORD till now: "+str(physical_nodes))
            node = int(input("Which node do you want to delete? "))
            success = delete_node(physical_nodes, finger_table, node)
            if success == -2:
                print("\n\nSorry, "+str(node) +" does not exist in the physical nodes list of the CHORD. Please try another node.")
            else:
                finger_table = success
                physical_nodes.sort()
                print('\n' + str(node)+" successfully deleted!")
                print("\nPhysical nodes in CHORD till now: "+str(physical_nodes))
        elif choice == 3:
            print("*"*50)
            print("\nPhysical nodes in CHORD till now: "+str(physical_nodes))
            print("\n\nYou can choose any node to lookup between "+str(init_node) + " and "+str(2**m-1) + " inclusive")
            node = int(input("Which node do you want to lookup? "))
            print("*"*50)
            success = lookup(physical_nodes, finger_table, node)
            if success == 0:
                print(str(node)+" is not in the CHORD.")
            elif success == -1:
                print("\n\nSorry, "+str(node) + " must be between "+str(init_node) + " and "+ str(2**m-1) + " inclusive.")
                continue
        elif choice == 4:
            print("*"*50)
            print("Showing CHORD...")
            print("You have to close the tkinter window to enter a choice again.")
            show(physical_nodes)
        elif choice == 5:
            print("*"*50)
            print("Program exited successfully.\n\n\n")
            break

This is a CHORD system. It is an example of Structured Peer-to-Peer architecture.

In this implementation, you can add, delete, lookup any physical/logical node, or view the CHORD.

As per instruction, initially there is one node in the CHORD (which is 0).

You can delete any physical node from the CHORD till there is at least one node left.

In the CHORD visualization, the green nodes are the physical nodes.

Following is the instruction for performing add/delete/lookup operations.
Let's start!


**************************************************
Press:
1 Add a random node
2 Delete a node
3 Lookup any node
4 View CHORD
5 Exit the program
**************************************************
Enter your choice: 
1
**************************************************
Randomly selected node in between 0 and 31 is: 6

6 successfully added to the CHORD!

Physical nodes in CHORD till now: [0, 6]

**************************************************
Press:
1 Add a random node
2 Delete a node
3 Looku

1
**************************************************
Randomly selected node in between 0 and 31 is: 8

8 successfully added to the CHORD!

Physical nodes in CHORD till now: [1, 4, 6, 8, 9, 10, 11, 12, 18, 22, 27, 29, 31]

**************************************************
Press:
1 Add a random node
2 Delete a node
3 Lookup any node
4 View CHORD
5 Exit the program
**************************************************
Enter your choice: 
1
**************************************************
Randomly selected node in between 0 and 31 is: 12


Randomly selected already exists in the CHORD. Try again.

**************************************************
Press:
1 Add a random node
2 Delete a node
3 Lookup any node
4 View CHORD
5 Exit the program
**************************************************
Enter your choice: 
1
**************************************************
Randomly selected node in between 0 and 31 is: 26

26 successfully added to the CHORD!

Physical nodes in CHORD till now: [1, 4, 6,

# 